In [ ]:
from datasets import load_dataset
from PIL import Image   

ds = load_dataset("detection-datasets/fashionpedia")


In [ ]:
for idx,image in enumerate(ds['val'][:100]["image"]):
    if isinstance(image, Image.Image):
        # Define the path to save the image
        image_path = f"fashion\\fashion_image_{idx}.jpg"  # Change the filename as needed

        # Save the image
        image.save(image_path)
        print(f'Saved image to {image_path}')

In [15]:
from fastAPI.services.pix import Pixtral
model = Pixtral(api_key="XXXXXX",
                 model_name="pixtral-12b-2409")

In [17]:
prompt = """
Analyze the given image of clothing and generate a detailed description that includes the following aspects:

Color: Describe the predominant colors present in the clothing item, including any patterns or combinations.

Length: Indicate the length of the garment (e.g., full-length, knee-length, cropped, etc.).

Textile: Identify the fabric or material of the clothing, such as cotton, silk, denim, etc.

Fitting: Describe the fit of the garment (e.g., loose, fitted, oversized, tailored).

Styles: Mention the style of the clothing (e.g., casual, formal, sporty, bohemian, etc.).

Textile Patterns: Include any patterns or prints present on the textile (e.g., floral, striped, polka dots).

Ocassion: Specify the occasion of the clothing (e.g., work, formal, casual, party etc.)

companions: With whom the person should wear the clothing.

Please provide a comprehensive and vivid description based on these aspects, highlighting unique features and characteristics of the clothing item shown in the image."""

In [13]:
image_encoding = model.file_to_data_url("fashion\\fashion_image_0.jpg")
response = model.complete(prompt,image_base46_encoding=image_encoding)
response.dict().get("choices")[0].get("message").get("content")

In [21]:
import pandas as pd
results = []

In [1]:
from fastAPI.services.faiss_db import DenseSearchFAISS
import pandas as pd

for idx in range(192,193):
    
    # Save the image temporarily for encoding
    temp_image_path = f"men_fashion\\fashion_image_{idx}.jpg"
    
    # Encode the image to base64
    image_encoding = model.file_to_data_url(temp_image_path)

    # Generate the description using your model
    response = model.complete(prompt,
                              image_base46_encoding=image_encoding)

    # Extract the content
    description = response.dict().get("choices")[0].get("message").get("content")
    
    # Store the results
    results.append({
        "image_filename": temp_image_path,
        "description": description
    })

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)

    # Save the results to a CSV file
    results_df.to_csv("men_fashion_descriptions3.csv", index=False)
    print(f"Descriptions saved to fashion_descriptions {idx}.csv")

In [2]:
db = DenseSearchFAISS(embedding_model="mistral-embed",
                      api_key="XXXXXXXXXXX",
                     dimension=1024,
                     base_path=r"C:\Users\saurav\OneDrive - URV\Escritorio\Hackathon\images",
                     prefix="fashion_image",
                     suffix="jpg"
                     )

FAISS index and IDs loaded from disk.


In [5]:
# Load data
df = pd.read_csv("fashion_descriptions.csv")

In [7]:
# Add documents to the FAISS index in batches
descriptions = df["description"].tolist()
ids = list(range(0, 100))

In [8]:
# Function to chunk the list
def chunk_list(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

In [ ]:
batch_size = 5
description_batches = list(chunk_list(descriptions, batch_size))
id_batches = list(chunk_list(ids, batch_size))

for desc_batch, id_batch in zip(description_batches, id_batches):
    db.add_documents(documents=desc_batch, ids=id_batch)
    print(f"Added batch: {id_batch}")


In [3]:
query = "I want to buy a green pant can you show, which is suitable for casual wear. it should be for man"
res = db.search(query, k=5,encoded=True)